In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import altair as alt

In [2]:
nba_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_train.csv')
nba_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_test.csv')

First we will fit a simple logistic model with an L1 penalty. This will encourage sparsity for our coefficients, aiding in feature selection.

First we can examine the proportions of All-NBA in our dataset. We initially created a train-test split where we attempt to create similar distributions for each set.

In [3]:
#Proportions of all_nba_c_year
nba_train['all_nba_c_year'].value_counts(normalize=True)

0    0.967255
1    0.032745
Name: all_nba_c_year, dtype: float64

In [4]:
nba_test['all_nba_c_year'].value_counts(normalize=True)

0    0.96727
1    0.03273
Name: all_nba_c_year, dtype: float64

Clearly we have an incredibly unbalanced dataset, but we may try to fit our models initially to see if we can improve upon them.

First we may try to filter our data based on variables like minutes played (`MP`), games played (`G`), since we know that these awards go to the best players in the NBA, and the best players tend to play a lot. In the new 2023 CBA (collective bargaining agreement), there is a minimum game requirement (65 games) that must be met in order to win All-NBA. However, since this rule was not in place for prior awards (where this data comes from), we can instead filter so that we only consider players who have played more games than the players with the least minutes played and games played that still won All-NBA.

In [5]:
min_minutes = nba_train[(nba_train['all_nba_c_year']==1)].MP.min()
min_G = nba_train[(nba_train['all_nba_c_year']==1)].G.min()
nba_filt_train = nba_train[(nba_train['MP']>=min_minutes) & (nba_train['G']>=min_G)]
nba_filt_test = nba_test[(nba_test['MP']>=min_minutes) & (nba_test['G']>=min_G)]




y_train = nba_filt_train['all_nba_c_year']

y_test = nba_filt_test['all_nba_c_year']

First we may fit a simpler model. Which players will make all-nba, versus which players won't? In this case we will ignore teams and instead only focus on the binary indicator. We will also see how the classifier does when we don't filter for positions

We will construct this model using a current year players stats and predict whether they make all_nba in the current year.

Since we have already standardized our variables, we must simply one hot encode the categorical variable, `Tm`. We will use a pipeline for ease of use.

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

num_features = ['Age','G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'W',
       'num_all_nba']

cat_features = ['Tm']

#Now I will create a pipeline where I extract my_features, and apply OHE to cat_features
ct = ColumnTransformer(
    [("select", "passthrough", num_features),
     ("ohe", OneHotEncoder(handle_unknown="ignore"), cat_features)],
     remainder="drop"
)



clf = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])

param_grid = {'classifier__C': np.logspace(-4, 4, num=10)}
model = GridSearchCV(clf, param_grid)



In [7]:
model.fit(nba_filt_train, y_train)


GridSearchCV(estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('select',
                                                                         'passthrough',
                                                                         ['Age',
                                                                          'G',
                                                                          'GS',
                                                                          'MP',
                                                                          'FG',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P',
                                                                          '3PA',
                                                                          '3P%',
                                                                          '2P',
                                                                          '2PA',
                                                                          '2P%',
                                                                          'eFG%',
                                                                          'FT',
                                                                          'FTA',
                                                                          'FT%',
                                                                          'ORB',
                                                                          'DRB',
                                                                          'TRB',
                                                                          'AST',
                                                                          'STL',
                                                                          'BLK',
                                                                          'TOV',
                                                                          'PF',
                                                                          'PTS',
                                                                          'PER',
                                                                          'TS%',
                                                                          '3PAr',
                                                                          'FTr', ...]),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Tm'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='l1',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])})

We see that doing this logistic regression, with an L1 penalty (to encourage sparsity), we fit our testing set with the following metrics:

In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(nba_filt_test)))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       819
           1       0.82      0.75      0.78       123

    accuracy                           0.95       942
   macro avg       0.89      0.86      0.88       942
weighted avg       0.94      0.95      0.94       942



We see that our model fit relatively well, even with the unbalanced dataset.

Looking at a cross table for our predictions and true data we see:

In [9]:
pd.crosstab(y_test, model.predict(nba_filt_test), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,799,20
1,31,92


We see our L1 penalty reduced the number of covariates from 79 to 34 

In [10]:
#Now we will extract the feature names from the pipeline
feature_names = model.best_estimator_.named_steps['col_transform'].get_feature_names_out()
coef_df = pd.DataFrame({'coef':model.best_estimator_['classifier'].coef_[0]
                        ,'var':feature_names})
coef_df.shape

(79, 2)

In [11]:
coef_df_nz = coef_df[coef_df['coef']!=0]
coef_df_nz.shape

(34, 2)

We see from our coefficient plot that FGA seems to have the highest impact on the log-odds of winning All-NBA, while FT% has the biggest negative impact. Surprisingly we also see that usage % is another of the strongest negative covariates. This bears further study, but for the purposes of this project (prediction), we do not necessarily care about how the model is weighing each of the features

In [12]:
#Now we will make a bar chart of these coefficients
alt.Chart(coef_df_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

alt.Chart(...)

We see for this dataset we have the following players who were predicted All-NBA but did not win it. (False Positives)

In [13]:
nba_filt_test[(model.predict(nba_filt_test)==1) & (y_test!=1)][['Player',"year"]]

,Player,year
463,Ja Morant,2023
577,James Harden,2023
794,Trae Young,2021
851,James Harden,2021
1152,Chauncey Billups,2008
1187,Kevin Johnson,1997
1803,Horace Grant,1992
1951,Andrei Kirilenko,2004
1995,Karl Malone,2003
2285,Josh Smith,2012


For our False Negatives we have:

In [14]:
nba_filt_test[(model.predict(nba_filt_test)!=1) & (y_test==1)][['Player',"year"]]

,Player,year
23,Gary Payton,1994
284,Joe Dumars,1990
304,Joe Dumars,1993
314,Mitch Richmond,1996
652,Mitch Richmond,1997
1010,Chauncey Billups,2009
1055,Tim Hardaway,1993
1079,Manu Ginóbili,2011
1093,Stephon Marbury,2003
1247,Reggie Miller,1996


Now we may consider fitting position specific models. We will use the same `MP` and `G` filters we used prior.

In [15]:
nba_g_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_g_train.csv')
nba_g_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_g_test.csv')

In [16]:
nba_g_filt_train = nba_g_train[(nba_g_train['MP']>=min_minutes) & (nba_g_train['G']>=min_G)]
nba_g_filt_test = nba_g_test[(nba_g_test['MP']>=min_minutes) & (nba_g_test['G']>=min_G)]
y_g_train = nba_g_filt_train['all_nba_c_year']
y_g_test = nba_g_filt_test['all_nba_c_year']


In [17]:
#Now I will create my pipeline as before, but I will include a step to remove a subset of variables I specify
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


clf_g = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])

model_g = GridSearchCV(clf_g, param_grid)
model_g.fit(nba_g_filt_train, y_g_train)

GridSearchCV(estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('select',
                                                                         'passthrough',
                                                                         ['Age',
                                                                          'G',
                                                                          'GS',
                                                                          'MP',
                                                                          'FG',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P',
                                                                          '3PA',
                                                                          '3P%',
                                                                          '2P',
                                                                          '2PA',
                                                                          '2P%',
                                                                          'eFG%',
                                                                          'FT',
                                                                          'FTA',
                                                                          'FT%',
                                                                          'ORB',
                                                                          'DRB',
                                                                          'TRB',
                                                                          'AST',
                                                                          'STL',
                                                                          'BLK',
                                                                          'TOV',
                                                                          'PF',
                                                                          'PTS',
                                                                          'PER',
                                                                          'TS%',
                                                                          '3PAr',
                                                                          'FTr', ...]),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Tm'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='l1',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])})

By looking at only guards we have similar accuracy, with worse recall for true positives

In [18]:
print(classification_report(y_g_test, model_g.predict(nba_g_filt_test)))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       369
           1       0.80      0.65      0.72        49

    accuracy                           0.94       418
   macro avg       0.88      0.82      0.84       418
weighted avg       0.94      0.94      0.94       418



In [19]:
pd.crosstab(y_g_test, model_g.predict(nba_g_filt_test), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,361,8
1,17,32


We have our false positives as:

In [20]:
nba_g_filt_test[(model_g.predict(nba_g_filt_test)==1) & (y_g_test!=1)][['Player','year']]

,Player,year
463,Ja Morant,2023
577,James Harden,2023
774,Jimmy Butler,2015
794,Trae Young,2021
851,James Harden,2021
1152,Chauncey Billups,2008
1187,Kevin Johnson,1997
1250,John Stockton,2000


We have our false negatives as:

In [21]:
nba_g_filt_test[(model_g.predict(nba_g_filt_test)!=1) & (y_g_test==1)][['Player','year']]

,Player,year
23,Gary Payton,1994
284,Joe Dumars,1990
304,Joe Dumars,1993
314,Mitch Richmond,1996
652,Mitch Richmond,1997
673,Deron Williams,2010
717,Isiah Thomas,1987
1010,Chauncey Billups,2009
1055,Tim Hardaway,1993
1079,Manu Ginóbili,2011


Examining our coefficients we see the L1 penalty reduced our number of features by almost 60.

In [22]:
coef_g_df = pd.DataFrame({'coef':model_g.best_estimator_['classifier'].coef_[0],
                          'var':feature_names})
coef_df.shape

(79, 2)

In [23]:
coef_g_nz = coef_g_df[coef_g_df['coef']!=0]
coef_g_nz.shape

(20, 2)

Looking at their values we have:

In [24]:
alt.Chart(coef_g_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

alt.Chart(...)

We see here that VORP has one of the largest positive coefficients, while number of personal fouls is one of the largest magnitude negative coefficients. 

Repeating this analysis for the C's and F's we get:

F's:

In [25]:
nba_F_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_F_train.csv')
nba_F_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_F_test.csv')

nba_F_filt_train = nba_F_train[(nba_F_train['MP']>=min_minutes) & (nba_F_train['G']>=min_G)]
nba_F_filt_test = nba_F_test[(nba_F_test['MP']>=min_minutes) & (nba_F_test['G']>=min_G)]


y_F_train = nba_F_filt_train['all_nba_c_year']
y_F_test = nba_F_filt_test['all_nba_c_year']

In [26]:
clf_F = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_F = GridSearchCV(clf_F, param_grid)
model_F.fit(nba_F_filt_train, y_F_train)

GridSearchCV(estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('select',
                                                                         'passthrough',
                                                                         ['Age',
                                                                          'G',
                                                                          'GS',
                                                                          'MP',
                                                                          'FG',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P',
                                                                          '3PA',
                                                                          '3P%',
                                                                          '2P',
                                                                          '2PA',
                                                                          '2P%',
                                                                          'eFG%',
                                                                          'FT',
                                                                          'FTA',
                                                                          'FT%',
                                                                          'ORB',
                                                                          'DRB',
                                                                          'TRB',
                                                                          'AST',
                                                                          'STL',
                                                                          'BLK',
                                                                          'TOV',
                                                                          'PF',
                                                                          'PTS',
                                                                          'PER',
                                                                          'TS%',
                                                                          '3PAr',
                                                                          'FTr', ...]),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Tm'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='l1',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])})

In [27]:
print(classification_report(y_F_test, model_F.predict(nba_F_filt_test)))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97       325
           1       0.80      0.88      0.83        49

    accuracy                           0.95       374
   macro avg       0.89      0.92      0.90       374
weighted avg       0.96      0.95      0.96       374



In [28]:
pd.crosstab(y_F_test, model_F.predict(nba_F_filt_test), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,314,11
1,6,43


In [29]:
coef_F_df = pd.DataFrame({'coef':model_F.best_estimator_['classifier'].coef_[0],
                          'var':feature_names})
coef_F_df.shape

(79, 2)

In [30]:
coef_F_nz = coef_F_df[coef_F_df['coef']!=0]
coef_F_nz.shape

(53, 2)

In [31]:
alt.Chart(coef_F_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

alt.Chart(...)

C's:

In [32]:
nba_C_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_C_train.csv')
nba_C_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_C_test.csv')

nba_C_filt_train = nba_C_train[(nba_C_train['MP']>=min_minutes) & (nba_C_train['G']>=min_G)]
nba_C_filt_test = nba_C_test[(nba_C_test['MP']>=min_minutes) & (nba_C_test['G']>=min_G)]


y_C_train = nba_C_filt_train['all_nba_c_year']
y_C_test = nba_C_filt_test['all_nba_c_year']

clf_C = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_C = GridSearchCV(clf_C, param_grid)
model_C.fit(nba_C_filt_train, y_C_train)

GridSearchCV(estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('select',
                                                                         'passthrough',
                                                                         ['Age',
                                                                          'G',
                                                                          'GS',
                                                                          'MP',
                                                                          'FG',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P',
                                                                          '3PA',
                                                                          '3P%',
                                                                          '2P',
                                                                          '2PA',
                                                                          '2P%',
                                                                          'eFG%',
                                                                          'FT',
                                                                          'FTA',
                                                                          'FT%',
                                                                          'ORB',
                                                                          'DRB',
                                                                          'TRB',
                                                                          'AST',
                                                                          'STL',
                                                                          'BLK',
                                                                          'TOV',
                                                                          'PF',
                                                                          'PTS',
                                                                          'PER',
                                                                          'TS%',
                                                                          '3PAr',
                                                                          'FTr', ...]),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Tm'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='l1',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])})

We see the Centers had a quite poor positive recall in this dataset. This makes sense, since this dataset is the most unbalanced of the three, since only one center is picked for each All-NBA team.

In [33]:
print(classification_report(y_C_test, model_C.predict(nba_C_filt_test)))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95       125
           1       0.79      0.60      0.68        25

    accuracy                           0.91       150
   macro avg       0.86      0.78      0.81       150
weighted avg       0.90      0.91      0.90       150



In [34]:
pd.crosstab(y_C_test, model_C.predict(nba_C_filt_test), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,121,4
1,10,15


In [35]:
coef_C = pd.DataFrame({'coef':model_C.best_estimator_['classifier'].coef_[0],
                       'var':feature_names})
coef_C.shape

(79, 2)

In [36]:
coef_C_nz = coef_C[coef_C['coef']!=0]
coef_C_nz.shape

(19, 2)

In [37]:
alt.Chart(coef_C_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

alt.Chart(...)

For this model Wins and the advanced statistic Win shares were two of the stats with the largest positive impact on the model, while FT% was one of the largest negative coefficients

Now we may try to balance our datasets. First we will try this on the larger dataset. We will first use SMOTENC sampling (Synthetic Minority Oversampling Technique-Numerical, Categorical). This method uses a k-nearest neighbors approach (default k=5).

In [38]:
#balance all_nba_c_year in the nba_train set
from imblearn.over_sampling import SMOTENC

smote = SMOTENC(random_state=0,categorical_features=[nba_filt_train[num_features+cat_features].shape[1]-1])
X_train_resampled, y_train_resampled = smote.fit_resample(nba_filt_train[num_features+cat_features], nba_filt_train['all_nba_c_year'])

In [39]:
clf_bal = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_smote = GridSearchCV(clf_bal, param_grid)
model_smote.fit(X_train_resampled, y_train_resampled)

GridSearchCV(estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('select',
                                                                         'passthrough',
                                                                         ['Age',
                                                                          'G',
                                                                          'GS',
                                                                          'MP',
                                                                          'FG',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P',
                                                                          '3PA',
                                                                          '3P%',
                                                                          '2P',
                                                                          '2PA',
                                                                          '2P%',
                                                                          'eFG%',
                                                                          'FT',
                                                                          'FTA',
                                                                          'FT%',
                                                                          'ORB',
                                                                          'DRB',
                                                                          'TRB',
                                                                          'AST',
                                                                          'STL',
                                                                          'BLK',
                                                                          'TOV',
                                                                          'PF',
                                                                          'PTS',
                                                                          'PER',
                                                                          'TS%',
                                                                          '3PAr',
                                                                          'FTr', ...]),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Tm'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='l1',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])})

In [40]:
print(classification_report(nba_filt_test['all_nba_c_year'], model_smote.predict(nba_filt_test[num_features+cat_features])))

              precision    recall  f1-score   support

           0       0.98      0.92      0.95       819
           1       0.62      0.90      0.74       123

    accuracy                           0.92       942
   macro avg       0.80      0.91      0.84       942
weighted avg       0.94      0.92      0.92       942



In [41]:
pd.crosstab(nba_filt_test['all_nba_c_year'], model_smote.predict(nba_filt_test[num_features+cat_features]), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,752,67
1,12,111


We see using this balanced dataset greatly increases our recall, but at the cost of our precision. 

We may also try under-sampling. This will essentially remove rows of the majority class (those who did not make All-NBA).

In [42]:
#Now we may use undersampling to balance the classes
from imblearn.under_sampling import RandomUnderSampler
ran_uns = RandomUnderSampler(random_state=0)
X_train_resampled, y_train_resampled = ran_uns.fit_resample(nba_filt_train[num_features+cat_features], nba_filt_train['all_nba_c_year'])


In [43]:
clf_us_bal = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_us = GridSearchCV(clf_us_bal, param_grid)
model_us.fit(X_train_resampled, y_train_resampled)

GridSearchCV(estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('select',
                                                                         'passthrough',
                                                                         ['Age',
                                                                          'G',
                                                                          'GS',
                                                                          'MP',
                                                                          'FG',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P',
                                                                          '3PA',
                                                                          '3P%',
                                                                          '2P',
                                                                          '2PA',
                                                                          '2P%',
                                                                          'eFG%',
                                                                          'FT',
                                                                          'FTA',
                                                                          'FT%',
                                                                          'ORB',
                                                                          'DRB',
                                                                          'TRB',
                                                                          'AST',
                                                                          'STL',
                                                                          'BLK',
                                                                          'TOV',
                                                                          'PF',
                                                                          'PTS',
                                                                          'PER',
                                                                          'TS%',
                                                                          '3PAr',
                                                                          'FTr', ...]),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Tm'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='l1',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])})

In [44]:
print(classification_report(nba_filt_test['all_nba_c_year'], model_us.predict(nba_filt_test[num_features+cat_features])))

              precision    recall  f1-score   support

           0       0.99      0.90      0.94       819
           1       0.59      0.93      0.72       123

    accuracy                           0.91       942
   macro avg       0.79      0.91      0.83       942
weighted avg       0.94      0.91      0.91       942



In [45]:
pd.crosstab(nba_filt_test['all_nba_c_year'], model_us.predict(nba_filt_test[num_features+cat_features]), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,739,80
1,9,114


Finally we can try over-sampling.

In [46]:
#Now we may use oversampling to balance the classes
from imblearn.over_sampling import RandomOverSampler
ran_os = RandomOverSampler(random_state=0)
X_train_resampled, y_train_resampled = ran_os.fit_resample(nba_filt_train[num_features+cat_features], nba_filt_train['all_nba_c_year'])
clf_os_bal = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_os = GridSearchCV(clf_os_bal, param_grid)
model_os.fit(X_train_resampled, y_train_resampled)

GridSearchCV(estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('select',
                                                                         'passthrough',
                                                                         ['Age',
                                                                          'G',
                                                                          'GS',
                                                                          'MP',
                                                                          'FG',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P',
                                                                          '3PA',
                                                                          '3P%',
                                                                          '2P',
                                                                          '2PA',
                                                                          '2P%',
                                                                          'eFG%',
                                                                          'FT',
                                                                          'FTA',
                                                                          'FT%',
                                                                          'ORB',
                                                                          'DRB',
                                                                          'TRB',
                                                                          'AST',
                                                                          'STL',
                                                                          'BLK',
                                                                          'TOV',
                                                                          'PF',
                                                                          'PTS',
                                                                          'PER',
                                                                          'TS%',
                                                                          '3PAr',
                                                                          'FTr', ...]),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Tm'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='l1',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])})

In [47]:
print(classification_report(nba_filt_test['all_nba_c_year'], model_os.predict(nba_filt_test[num_features+cat_features])))

              precision    recall  f1-score   support

           0       0.98      0.91      0.95       819
           1       0.61      0.90      0.73       123

    accuracy                           0.91       942
   macro avg       0.80      0.91      0.84       942
weighted avg       0.94      0.91      0.92       942



In [48]:
pd.crosstab(nba_filt_test['all_nba_c_year'], model_os.predict(nba_filt_test[num_features+cat_features]), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,749,70
1,12,111


We see all of these 3 methods are not significantly better, or are quite worse than the baseline unbalanced class model. Specifically we have quite poor precision for our positive classifications 